In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
import sklearn.utils
import sklearn.metrics
import iisignature
import torch
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from typing import List, Optional, Dict, Set, Callable, Any
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets
import sigkernel
import scipy
from scipy.interpolate import interp1d
from numba import njit

from signature import streams_to_sigs, transform_stream
from conformance import BaseclassConformanceScore, pairwise_kernel_gram, stream_to_torch
from kernels import linear_kernel_gram, rbf_kernel_gram, poly_kernel_gram
from kernels import integral_kernel_gram, sig_kernel_gram

# tslearn datasets (equal length)

* equal length (in time) UCR_UEA multivariate time series 

In [2]:
#experiment code

def print_dataset_stats(num_classes, d, T, N_train, N_test):
    print("Number of Classes:", num_classes)
    print("Dimension of path:", d)
    print("Length:", T)
    print("Train:", N_train)
    print("Test:", N_test)


def case_static(train:np.ndarray, 
                test:np.ndarray,
                static_kernel_gram:Callable,):
    """Calculates the gram matrices of equal length time series for 
    a static kernel on R^d. Train and test are of shape (N1, T, d) 
    and (N2, T, d). Static kernel should take in two arrays of shape 
    (M, T*d) and return the Gram matrix."""
    N1, T, d = train.shape
    N2, _, _ = test.shape
    train = train.reshape(N1, -1)
    test = test.reshape(N2, -1)
    vv_gram = static_kernel_gram(train, train)
    uv_gram = static_kernel_gram(test, train)
    return vv_gram, uv_gram


def case_linear(train:np.ndarray, 
                test:np.ndarray):
    """Calculates the gram matrices for the euclidean inner product.
    Train and test are of shape (N1, T, d) and (N2, T, d)."""
    return case_static(train, test, linear_kernel_gram)


def case_rbf(train:np.ndarray, 
             test:np.ndarray,
             sigma:float):
    """Calculates the gram matrices for the rbf kernel.
    Train and test are of shape (N1, T, d) and (N2, T, d)."""
    rbf_ker = lambda X, Y : rbf_kernel_gram(X, Y, sigma)
    return case_static(train, test, rbf_ker)


def case_poly(train:np.ndarray, 
              test:np.ndarray,
              p:float):
    """Calculates the gram matrices for the rbf kernel.
    Train and test are of shape (N1, T, d) and (N2, T, d)."""
    poly_ker = lambda X, Y : poly_kernel_gram(X, Y, p)
    return case_static(train, test, poly_ker)


def case_gak(train:List[np.ndarray], 
                   test:List[np.ndarray], 
                   fixed_length:bool,
                   sigma:float = 1.0,):
    """Calculates the gram matrices for the gak kernel.
    Train and test are lists of possibly variable length multidimension 
    time series of shape (T_i, d)"""
    #pick sigma parameter according to GAK paper
    if fixed_length:
        sigma = tslearn.metrics.sigma_gak(np.array(train))

    #compute gram matrices
    kernel = lambda s1, s2 : tslearn.metrics.gak(s1, s2, sigma)
    vv_gram = pairwise_kernel_gram(train, train, kernel, sym=True, disable_tqdm=False)
    uv_gram = pairwise_kernel_gram(test, train, kernel, sym=False, disable_tqdm=False)
    return vv_gram, uv_gram


# Solely to be used in sigkernel library. See e.g. sigkernel.LinearKernel.
# Had to reimplement it since the original class is missing the scalar in 
# the Gram method
class LinearKernel():
    def __init__(self, scale=1.0):
        self.scale = scale
        
    def batch_kernel(self, X, Y):
        return self.scale*torch.bmm(X, Y.permute(0,2,1))

    def Gram_matrix(self, X, Y):
        return self.scale * torch.einsum('ipk,jqk->ijpq', X, Y)
    
class PolyKernel():
    def __init__(self, scale=1.0, p=2):
        self.scale = scale
        self.p = p
        
    def batch_kernel(self, X, Y):
        return self.scale * (1+torch.bmm(X, Y.permute(0,2,1)))**self.p

    def Gram_matrix(self, X, Y):
        return self.scale * (1+torch.einsum('ipk,jqk->ijpq', X, Y))**self.p

 
def case_sig_pde(train:List[np.ndarray], 
                 test:List[np.ndarray], 
                 dyadic_order:int = 3,
                 static_kernel = sigkernel.LinearKernel(),
                ):
    """Calculates the signature kernel gram matrices of the train and test.
    Train and test are lists of possibly variable length multidimension 
    time series of shape (T_i, d)"""
    sig_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)
    kernel = lambda s1, s2 : sig_kernel.compute_kernel(
                                stream_to_torch(s1), 
                                stream_to_torch(s2)).numpy()[0]
    vv_gram = pairwise_kernel_gram(train, train, kernel, sym=True, disable_tqdm=False)
    uv_gram = pairwise_kernel_gram(test, train, kernel, sym=False, disable_tqdm=False)
    return vv_gram, uv_gram


def calc_grams(train:List[np.ndarray], 
               test:List[np.ndarray],
               param_dict:Dict[str, Any], # name : value
               fixed_length:bool, 
               ):   
    """Calculates gram matrices <train, train>, <test, train> given a kernel.
    Train and test are lists of possibly variable length multidimension time 
    series of shape (T_i, d)"""

    #Transform to array if possible
    if fixed_length:
        train = np.array(train)
        test = np.array(test)
    
    #choose method based on kernel name
    kernel_name = param_dict["kernel_name"]
    if kernel_name == "linear":
        return case_linear(train, test)
    
    elif kernel_name == "rbf":
        return case_rbf(train, test, param_dict["sigma"])
    
    elif kernel_name == "poly":
        return case_poly(train, test, param_dict["p"])

    elif kernel_name == "gak":
        return case_gak(train, test, fixed_length)

    elif kernel_name == "truncated sig":
        vv_gram = sig_kernel_gram(train, train, param_dict["order"], linear_kernel_gram, sym=True)
        uv_gram = sig_kernel_gram(test, train, param_dict["order"], linear_kernel_gram)
        return vv_gram, uv_gram
    
    elif kernel_name == "truncated sig rbf":
        ker = lambda X, Y: rbf_kernel_gram(X, Y, param_dict["sigma"])
        vv_gram = sig_kernel_gram(train, train, param_dict["order"], ker, sym=True)
        uv_gram = sig_kernel_gram(test, train, param_dict["order"], ker)
        return vv_gram, uv_gram
    
    elif kernel_name == "truncated sig poly":
        ker = lambda X, Y : poly_kernel_gram(X, Y, param_dict["p"])
        vv_gram = sig_kernel_gram(train, train, param_dict["order"], ker, sym=True)
        uv_gram = sig_kernel_gram(test, train, param_dict["order"], ker)
        return vv_gram, uv_gram
    
    elif kernel_name == "signature pde":
        return case_sig_pde(train, 
                        test,
                        static_kernel=LinearKernel(1/train[0].shape[-1]),)
    
    elif kernel_name == "signature pde rbf":
        return case_sig_pde(train, 
                        test,
                        static_kernel=sigkernel.RBFKernel(
                            param_dict["sigma"] * train[0].shape[-1]),)

    elif kernel_name == "signature pde poly":
        return case_sig_pde(train, 
                        test,
                        static_kernel=PolyKernel(
                            1/train[0].shape[-1], param_dict["p"]),)
    
    elif kernel_name == "integral linear":
        vv_gram = integral_kernel_gram(train, train, linear_kernel_gram, fixed_length, sym=True)
        uv_gram = integral_kernel_gram(test, train, linear_kernel_gram, fixed_length)
        return vv_gram, uv_gram

    elif kernel_name == "integral rbf":
        ker = lambda X, Y, diag: rbf_kernel_gram(X, Y, param_dict["sigma"], diag)
        vv_gram = integral_kernel_gram(train, train, ker, fixed_length, sym=True)
        uv_gram = integral_kernel_gram(test, train, ker, fixed_length)
        return vv_gram, uv_gram

    elif kernel_name == "integral poly":
        ker = lambda X, Y, diag : poly_kernel_gram(X, Y, param_dict["p"], diag)
        vv_gram = integral_kernel_gram(train, train, ker, fixed_length, sym=True)
        uv_gram = integral_kernel_gram(test, train, ker, fixed_length)
        return vv_gram, uv_gram
    
    else:
        raise ValueError("Invalid kernel name:", kernel_name)


def normalize_streams(train:np.ndarray, 
                      test:np.ndarray,
                      ):
    """Inputs are 3D arrays of shape (N, T, d) where N is the number of time series, 
    T is the length of each time series, and d is the dimension of each time series."""
    # Normalize data by training set mean and std
    mean = np.mean(train, axis=0, keepdims=True)
    std = np.std(train, axis=0, keepdims=True)
    train = (train - mean) / std
    test = (test - mean) / std
    return train, test


def compute_aucs(distances_conf:np.ndarray,     #size N
                 distances_mahal:np.ndarray,    #size Nint
                 y_test:np.array,               #size N
                 class_to_test,):
    # 2 methods (conf, mahal), 2 metrics (roc_auc, pr_auc)
    aucs = np.zeros( (2, 2) ) 

    # Calculate one vs rest AUC, weighted by size of class
    for idx_conf_mahal, distances in enumerate([distances_conf, distances_mahal]):
        ovr_labels = y_test != class_to_test
        average="macro" #average = "macro" or "eeighted"
        roc_auc = sklearn.metrics.roc_auc_score(ovr_labels, distances, average=average)
        pr_auc = sklearn.metrics.average_precision_score(ovr_labels, distances, average=average)
        aucs[idx_conf_mahal, 0] = roc_auc
        aucs[idx_conf_mahal, 1] = pr_auc
    
    return aucs


def run_single_kernel_single_label(
        X_train:List[np.ndarray],
        y_train:np.ndarray,
        X_test:List[np.ndarray], 
        y_test:np.ndarray,
        class_to_test,
        param_dict:Dict[str, Any], # name : value
        fixed_length:bool,
        SVD_threshold:float = 10e-14,
        SVD_max_rank:Optional[int] = None,
        verbose:bool = False,
        vv_gram=None,
        uv_gram=None,
        return_all_levels:bool = False,

        ):
    """Computes the AUC scores (weighted one vs rest) for a single kernel,
    using kernelized nearest neighbour variance adjusted distances.

    Args:
        X_train (List[np.ndarray]): List of time series of shape (T_i, d).
        y_train (np.array): 1-dim array of class labels.
        X_test (List[np.ndarray]): List of time series of shape (T_i, d).
        y_test (np.array): 1-dim array of class labels.
        unique_labels (np.array): Array of unique class labels.
        kernel_name (str): Name of the kernel to use.
        fixed_length (bool): If True, uses the optimized kernels for fixed 
                             length time series.
        normalize (bool): If True, normalizes train and test by the training set
                          mean and std.
        dyadic_order (int): Dyadic order for PDE solver 
                            (int > 0, higher = more accurate but slower).
        max_batch (int): Batch size in sig kernel computations.
        trunc_sig_dim_bound (int): Upper bound on the dimensionality of the 
                                  truncated signature.
        SVD_threshold (float): Sets all eigenvalues below this threshold to be 0.
        SVD_max_rank (int): Sets all SVD eigenvalues to be 0 beyond 'SVD_max_rank'.
    """
    # Get all samples of the current class
    idxs = np.where(y_train == class_to_test)[0]
    corpus = [X_train[k] for k in idxs]
    test = X_test
    if fixed_length:
        corpus, test = normalize_streams(np.array(corpus), test)
    
    # Calculate amomaly distancce scores for all test samples
    if (vv_gram is None) and (uv_gram is None):
        vv_gram, uv_gram = calc_grams(corpus, test, param_dict, fixed_length)
    scorer = BaseclassConformanceScore(vv_gram, SVD_threshold, verbose=verbose, 
                                    SVD_max_rank=SVD_max_rank)
        
    # only return accs for highest allowed threshold
    if not return_all_levels:
        conf, mahal = scorer._anomaly_distance(uv_gram, method="both")
        aucs = compute_aucs(conf, mahal, y_test, class_to_test)
    else:
        conf, mahal = scorer._anomaly_distance(uv_gram, method="both",
                                                return_all_levels=True)
        aucs = np.array([compute_aucs(c, m, y_test, class_to_test)
                         for c,m in zip(conf.T, mahal.T)])

    return aucs


def run_all_kernels(X_train:List[np.ndarray], 
                    y_train:np.array, 
                    X_test:List[np.ndarray], 
                    y_test:np.array, 
                    unique_labels:np.array, 
                    kernel_names:List[str],
                    fixed_length:bool,
                    verbose:bool = True,
                    ):
    kernel_results = {}
    for kernel_name in kernel_names:
        # 2 methods (conf, mahal), 2 metrics (roc_auc, pr_auc), C classes
        aucs = np.zeros( (2, 2, len(unique_labels)) ) 
        for i, label in enumerate(unique_labels):
            #run model
            scores = run_single_kernel_single_label(X_train, y_train, 
                                    X_test, y_test,label, kernel_name, #TODO no more kernel name
                                    fixed_length, verbose=verbose)
            aucs[:,:, i] = scores
        
        #update kernel results
        kernel_results[kernel_name] = aucs
    return kernel_results


def run_tslearn_experiments(dataset_names:List[str], 
                            kernel_names:List[str],
                            verbose:bool=True,
                            ):
    """Runs a series of time series anomaly detection experiments on the specified 
    tslearn datasets using kernel conformance scores."""
    experiments = {}
    for dataset_name in dataset_names:
        # Load dataset
        X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)

        # stats
        unique_labels = np.unique(y_train)
        num_classes = len(unique_labels)
        N_train, T, d = X_train.shape
        N_test, _, _  = X_test.shape
        print_dataset_stats(num_classes, d, T, N_train, N_test)

        # Run each kernel
        kernel_results = run_all_kernels(X_train, y_train, X_test, y_test, 
                                         unique_labels, kernel_names,
                                         fixed_length=True,
                                         verbose=verbose)
        
        #log dataset experiment
        experiments[dataset_name] = {"results": kernel_results, 
                                     "num_classes": num_classes, 
                                     "dim":d,
                                     "ts_length":T, 
                                     "N_train":N_train, 
                                     "N_test":N_test}
    return experiments

In [3]:
# #run experiments

# experiments = run_tslearn_experiments(
#     dataset_names = [
#         #'ArticularyWordRecognition', 
#         #'BasicMotions', 
#         #'Cricket',
#          ##########'ERing', #cant find dataset
#         'Libras', 
#         #'NATOPS', 
#         #'RacketSports',     
#         #'FingerMovements',
#         #'Heartbeat',
#         #'SelfRegulationSCP1', 
#         #'UWaveGestureLibrary'
#         ],
#     kernel_names = [
#         "linear",
#         #"rbf",
#         #"poly",
#         #"gak",
#         #"truncated sig",
#         #"truncated sig rbf",
#         #"truncated sig poly",
#         #"signature pde",
#         #"signature pde rbf",
#         #"signature pde poly",
#         #"integral linear",
#         #"integral rbf",
#         #"integral poly",
#         ],
#         verbose=True
#         )


# def print_experiment_results(experiments, round_digits=5):
#     for dataset_name, results in experiments.items():
#         #Dataset:
#         print("\nStart Dataset {dataset_name} results:", dataset_name)
#         print_dataset_stats(results["num_classes"], results["dim"], 
#                             results["ts_length"], results["N_train"], 
#                             results["N_test"])

#         #Results for each kernel:
#         for kernel_name, scores in results["results"].items():
#             print("\nKernel:", kernel_name)
#             scores = np.mean(scores, axis=2)
#             print("Conformance AUC:", round(scores[0, 0], round_digits))
#             print("Mahalanobis AUC:", round(scores[1, 0], round_digits))
#             print("Conformance PR AUC:", round(scores[0, 1], round_digits))
#             print("Mahalanobis PR AUC:", round(scores[1, 1], round_digits))

#         print("\nEnd Dataset {dataset_name} results\n\n\n")
        
# print_experiment_results(experiments)

In [4]:
#comments

# Kernel: truncated sig
# Conformance AUC: 0.90757
# Mahalanobis AUC: 0.90728
# Conformance PR AUC: 0.99236
# Mahalanobis PR AUC: 0.99235

# Kernel: linear
# Conformance AUC: 0.94646
# Mahalanobis AUC: 0.93737
# Conformance PR AUC: 0.99506
# Mahalanobis PR AUC: 0.99445

# Kernel: rbf
# Conformance AUC: 0.7671
# Mahalanobis AUC: 0.08571
# Conformance PR AUC: 0.95905
# Mahalanobis PR AUC: 0.84199

# Kernel: integral linear
# Conformance AUC: 0.94686
# Mahalanobis AUC: 0.93849
# Conformance PR AUC: 0.99512
# Mahalanobis PR AUC: 0.99451

# End Dataset {dataset_name} results

## Find all tslearn datasets

In [5]:
# _datasets = [
#             'ArticularyWordRecognition', 
#             'BasicMotions', 
#             'Cricket',
#             #'ERing',
#             'Libras', 
#             'NATOPS', 
#             'RacketSports',     
#             'FingerMovements',
#             'Heartbeat',
#             'SelfRegulationSCP1', 
#             'UWaveGestureLibrary'
#             ]

# import tslearn
# UCR_UEA_datasets = tslearn.datasets.UCR_UEA_datasets()

# for dataset_name in UCR_UEA_datasets.list_multivariate_datasets():
# #for dataset_name in _datasets:
#     print("Dataset:", dataset_name)
#     dataset = UCR_UEA_datasets.load_dataset(dataset_name)
#     if dataset[0] is not None:
#         X_train, y_train, X_test, y_test = dataset
#         num_classes = len(np.unique(y_train))
#         N_train, T, d = X_train.shape
#         N_test, _, _  = X_test.shape
        
#         print("Number of Classes:", num_classes)
#         print("Dimension of path:", d)
#         print("Length:", T)
#         print("Train Size, Test Size", N_train, N_test)
#         print()
#     else:
#         print("No dataset found")
#         print()

#yes
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: AtrialFibrillation
# No dataset found

#yes
# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: CharacterTrajectories
# No dataset found

#yes
# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: DuckDuckGeese
# No dataset found

# Dataset: EigenWorms
# Number of Classes: 5
# Dimension of path: 6
# Length: 17984
# Train Size, Test Size 128 131

#why not
# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train Size, Test Size 137 138

#longLength
# Dataset: EthanolConcentration
# Number of Classes: 4
# Dimension of path: 3
# Length: 1751
# Train Size, Test Size 261 263

# Dataset: ERing
# No dataset found

#big
# Dataset: FaceDetection
# Number of Classes: 2
# Dimension of path: 144
# Length: 62
# Train Size, Test Size 5890 3524

#yes
# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

#why not, maybe big length
# Dataset: HandMovementDirection
# Number of Classes: 4
# Dimension of path: 10
# Length: 400
# Train Size, Test Size 160 74

#smallTrain
# Dataset: Handwriting
# Number of Classes: 26
# Dimension of path: 3
# Length: 152
# Train Size, Test Size 150 850

#yes
# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

#big
# Dataset: InsectWingbeat
# Number of Classes: 10
# Dimension of path: 200
# Length: 22
# Train Size, Test Size 25000 25000

# Dataset: JapaneseVowels
# No dataset found

#yes
# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

#TODO I SHOULD INCLUDE
# Dataset: LSST
# Number of Classes: 14
# Dimension of path: 6
# Length: 36
# Train Size, Test Size 2459 2466

#length
# Dataset: MotorImagery
# Number of Classes: 2
# Dimension of path: 64
# Length: 3000
# Train Size, Test Size 278 100

#yes
# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

#TODO NOT TSLEARN. LENGTH WRONG
# Dataset: PenDigits
# Number of Classes: 10
# Dimension of path: 2
# Length: 8
# Train Size, Test Size 7494 3498

#highDim
# Dataset: PEMS-SF
# Number of Classes: 7
# Dimension of path: 963
# Length: 144
# Train Size, Test Size 267 173

#dim=1, big length
# Dataset: Phoneme
# Number of Classes: 39
# Dimension of path: 1
# Length: 1024
# Train Size, Test Size 214 1896

#yes
# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

#yes
# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: SelfRegulationSCP2
# Number of Classes: 2
# Dimension of path: 7
# Length: 1152
# Train Size, Test Size 200 180

# Dataset: SpokenArabicDigits
# No dataset found

#long, also very small set
# Dataset: StandWalkJump
# Number of Classes: 3
# Dimension of path: 4
# Length: 2500
# Train Size, Test Size 12 15

#yes
# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320


# Cross Validation code (for anomaly detection)

In [53]:
def repeat_k_folds(X:List,    #dataset
                y:np.ndarray, #class labels
                k:int = 5,
                n_repeats:int = 2,):
    repeats = [create_k_folds(X, y, k) for _ in range(n_repeats)]
    return repeats


def create_k_folds(X:List,    #dataset
                y:np.ndarray, #class labels
                k:int = 5,):
    """Generates balanced k-folds for cross-validation, where each fold
    is balanced the same as the original dataset."""
    #is X numpy array?
    is_numpy=True if isinstance(X, np.ndarray) else False

    #shuffle data
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    X = [X[i] for i in indices]
    y = np.array([y[i] for i in indices])
    unique_labels = np.unique(y)

    #split into classes
    classwise = {label:[] for label in unique_labels}
    for x, label in zip(X, y):
        classwise[label].append(x)

    #split into k-folds
    classwise_folds = {}
    for label, dataclass in classwise.items():
        classwise_folds[label] = np.array_split(dataclass, k)
    
    #create folds
    folds=[]
    for i in range(k):
        X_train = []
        y_train = []
        X_val = []
        y_val = []
        for label, dataclass in classwise_folds.items():
            for j in range(k):
                if j!=i:
                    X_train.extend(dataclass[j])
                    y_train.extend([label]*len(dataclass[j]))
                else:
                    X_val.extend(dataclass[j])
                    y_val.extend([label]*len(dataclass[j]))

        #convert to numpy if possible
        if is_numpy:
            X_train = np.array(X_train)
            X_val = np.array(X_val)
        y_train = np.array(y_train)
        y_val = np.array(y_val)
        folds.append([X_train, y_train, X_val, y_val])

    return folds


def get_hyperparam_combinations(kernel_name:str):
    """Returns a dict of hyperparameter ranges and a list of all 
    possible combinations of hyperparameters for the specified kernel"""
    ranges = get_hyperparam_ranges(kernel_name)
    values = ranges.values()
    keys = ranges.keys()

    if ranges:
        #create array of all combinations
        meshgrid = np.meshgrid(*values)
        combinations = np.vstack([x.flatten() for x in meshgrid]).T

        #convert to dict
        dict_combinations = [dict(zip(keys, vals)) for vals in combinations]

        return dict_combinations, ranges
    else:
        return [], {}


def get_hyperparam_ranges(kernel_name:str):
    """ Returns a dict of hyperparameter ranges for the specified kernel."""
    n_static_params = 7
    ranges = {}

    #static kernel params. Note that sig and integral kernels also use this
    if "rbf" in kernel_name:
        ranges["sigma"] = np.exp(np.linspace(-5, 0, n_static_params))
    elif "poly" in kernel_name:
        ranges["p"] = np.linspace(1.5, 3.5, n_static_params)

    return ranges
    


def evaluate_folds(folds:List[tuple],
                class_to_test,
                min_fold_size:int,
                fixed_length:bool,
                param_dict:Dict[str, Any], # name : value
                # ranges:Dict[str, np.array], # name : param range
                ):
    """"We permform anomaly detection using 'class_to_test' as the normal class. 
    We then calculate the AUC scores for the given hyperparameters."""
    scores = np.zeros((len(folds), min_fold_size))
    for i, fold in enumerate(folds):
        #for now does not support trunc sig. just add Max_trunc to param_dict
        X_train, y_train, X_val, y_val = fold
        aucs = run_single_kernel_single_label(X_train, y_train, X_val, y_val,
                                class_to_test, param_dict,
                                fixed_length, verbose=False,
                                return_all_levels=True,
                                SVD_threshold=0,
                                SVD_max_rank=min_fold_size, #TODO maybe change
                                )
        score = np.max(aucs, axis=1)# max of conf and mahal
        score = score[..., 0]       # only interested in roc (and not pr)
        #score = np.mean(score, axis=-1)# mean of roc and pr
        scores[i, :] = score[:min_fold_size]
    return np.mean(scores, axis=0) #average across folds



def cv_given_dataset(X:List,                #Training Dataset
                    y:np.array,            #Training class labels
                    unique_labels:np.array, #Unique class labels
                    kernel_names:List[str],
                    fixed_length:bool,
                    k:int = 5,          #k-fold cross validation
                    n_repeats:int = 2,  #repeats of k-fold CV
                    ):
    model_params_CV = {}
    # anomaly detection model is specified by a 
    # pair (normal class label, param_dict)
    # or rather (label, kernel_name, *params)

    # model_params_CV[pair] = CV_scores
    repeats_and_folds = repeat_k_folds(X, y, k, n_repeats)
    for kernel_name in kernel_names:
        #get hyperparams
        hyperparams, ranges = get_hyperparam_combinations(kernel_name)

        #loop over normal class
        for label in unique_labels:

            #calc minimum size of the label class
            min_fold_size = min([len(np.where(y_train==label)[0]) 
                                 for (_, y_train, _, _) in repeats_and_folds[0]])
            repeated_scores = np.zeros((n_repeats, len(hyperparams), min_fold_size))

            #loop over repeats and params
            for repeat_idx, folds in enumerate(repeats_and_folds):
                for i, param_dict in enumerate(hyperparams):
                    param_dict["kernel_name"] = kernel_name
                    param_dict["normal_class_label"] = label
                    scores = evaluate_folds(folds, label,
                                            min_fold_size, fixed_length,
                                            param_dict)
                    repeated_scores[repeat_idx, i] = scores
            
            #average scores
            print("repeated_scores", repeated_scores.shape)
            scores = np.mean(repeated_scores, axis=0) #average over repeats
            scores = np.max(scores, axis=0) #max over params
            best = np.argmax(scores)
            print("scores", scores.shape, scores)
            print("best", best)
            assert False


def test():
    X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset("Libras")
    unique_labels = np.unique(y_train)

    cv_given_dataset(X_train, 
                     y_train, 
                     unique_labels, 
                     ["linear"],
                     fixed_length=True)
test()

repeated_scores (2, 0, 9)


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
def get_hyperparam_combinations(kernel_name:str):
    """Returns a dict of hyperparameter ranges and a list of all 
    possible combinations of hyperparameters for the specified kernel"""
    ranges = get_hyperparam_ranges(kernel_name)
    values = ranges.values()
    meshgrid = np.meshgrid(*values)
    combinations = np.vstack([x.flatten() for x in meshgrid]).T
    return combinations, ranges


def get_hyperparam_ranges(kernel_name:str):
    """ Returns a dict of hyperparameter ranges for the specified kernel."""
    n_static_params = 9
    max_trunc_order = 15
    ranges = {}

    #static kernel params
    if "rbf" in kernel_name:
        ranges["sigma"] = np.exp(np.linspace(-5, 0, n_static_params))
    elif "poly" in kernel_name:
        ranges["p"] = np.linspace(1.5, 3.5, n_static_params)
    
    #trunc sig params
    if "truncated sig" in kernel_name:
        ranges["order"] = np.arange(1, max_trunc_order+1)
    return ranges

hyperparams, ranges = get_hyperparam_combinations("truncated sig rbf")

#print(hyperparams)

keys = ranges.keys()

better = [dict(zip(keys, vals)) for vals in hyperparams]
print(better)




In [ ]:
x, y = ranges.items()
print(x, "x\n")
print(y, "y\n")